In [33]:
import numpy as np
from collections import defaultdict
import random

In [3]:
class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.word_count = defaultdict(int)
        self.total_words = 0
        self.vocab_size = 0

    def build_vocab(self, sentences, min_count = 2):
        for word in sentences:
            self.word_count[word] += 1

        idx = 0
        for word, count in self.word_count.items():
            if count >= min_count:
                self.word2idx.update({word: idx})
                idx += 1
    
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
        self.total_words = sum([count for word, count in self.word_count.items() if count >= min_count])

    def word_to_index(self, word):
        return self.word2idx.get(word, -1)

    def index_to_word(self, index):
        return self.idx2word.get(index, None)

In [20]:
def generate_training_data(vocab, sentences, window_size = 2):
    training_data = []
    sentence_indices = [vocab.word_to_index(word) for word in sentences if vocab.word_to_index(word) != -1]

    for center_idx, center_word in enumerate(sentence_indices):
        context_start = max(0, center_idx - window_size)
        context_end = min(len(sentence_indices), center_idx + window_size + 1)

        for context_idx in range(context_start, context_end):
            if context_idx != center_idx:
                context_word = sentence_indices[context_idx]
                training_data.append((center_word, context_word))

    return np.array(training_data)

In [21]:
class Word2Vec:
    def __init__(self, vocab_size, embed_size = 100, learning_rate = 0.001):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.learning_rate = learning_rate

        self.W = np.random.uniform(-0.5, 0.5, (vocab_size, embed_size))
        self.W_prime = np.random.uniform(-0.5, 0.5, (embed_size, vocab_size))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x)

    def train(self, training_data, epochs = 1000):
        for epoch in range(epochs):
            loss = 0
            for center_word, context_word in training_data:
                h = self.W[center_word]
                u = np.dot(h, self.W_prime)
                y_pred = self.softmax(u)

                y_true = np.zeros(self.vocab_size)
                y_true[context_word] = 1

                error = y_pred - y_true

                self.W_prime -= self.learning_rate * np.outer(h, error)
                self.W[center_word] -= self.learning_rate * np.dot(self.W_prime, error)

                loss -= np.log(y_pred[context_word])

            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

In [22]:
with open('/kaggle/input/persianstopwords/persian.txt', 'r', encoding = 'utf-8') as f:
    stopwords = set(f.read().splitlines())

text = "ملکه و زن ها در کنار همسران و خانواده خود یعنی شاه و مرد ها در یک سرزمین پهناور زندگی می‌کردند شاه همیشه به مرد ها تذکر میداد که قدرت در اتحاد مرد ها و شاه نهفته است و در این قلمرو ملکه به زن ها یادآوری می‌کرد که همبستگی زن ها و ملکه مهم است و در این داستان هر مرد که نزد شاه یا زن که نزد ملکه می‌آمد از آنها حکم می‌گرفت تا به دیگران کمک کنند شاه عادل و قادر بود و ملکه خردمند و زیبا و هر مرد که از حکمت شاه یا زن که از عدالت ملکه راضی نبود نزد آنها می‌رفت تا شکایت خود را مطرح کند شاه و مرد و ملکه و زن در کنار هم بودند و هیچ کس از شاه یا ملکه نمی‌ترسید شاه همیشه به زبان میاورد که مرد ها باید به یکدیگر کمک کنند و ملکه تأکید داشتند زن ها هم باید متحد باشند"
words = text.split()
filtered_text = [word for word in words if word not in stopwords]
cleaned_text = ' '.join(filtered_text)
print(cleaned_text)

ملکه زن همسران خانواده شاه مرد سرزمین پهناور زندگی می‌کردند شاه مرد تذکر میداد قدرت اتحاد مرد شاه نهفته قلمرو ملکه زن یادآوری می‌کرد همبستگی زن ملکه مهم داستان مرد شاه زن ملکه می‌آمد حکم می‌گرفت کمک شاه عادل قادر ملکه خردمند زیبا مرد حکمت شاه زن عدالت ملکه راضی می‌رفت شکایت مطرح شاه مرد ملکه زن شاه ملکه نمی‌ترسید شاه زبان میاورد مرد کمک ملکه تأکید زن متحد


In [23]:
vocab = Vocabulary()
vocab.build_vocab(cleaned_text.split(' '))
training_data = generate_training_data(vocab, cleaned_text.split(' '))
word2vec_model = Word2Vec(vocab.vocab_size)
word2vec_model.train(training_data, epochs = 1000)

Epoch 0, Loss: 273.9280651175166
Epoch 100, Loss: 181.58500756594077
Epoch 200, Loss: 181.10085924976738
Epoch 300, Loss: 180.9460396128876
Epoch 400, Loss: 180.86920414258782
Epoch 500, Loss: 180.82340062796956
Epoch 600, Loss: 180.79319836412884
Epoch 700, Loss: 180.77191984707156
Epoch 800, Loss: 180.75620469921117
Epoch 900, Loss: 180.74418136837178


In [24]:
def get_word_embedding(word, vocab, model):
    word_idx = vocab.word_to_index(word)
    if word_idx != -1:
        return model.W[word_idx]
    else:
        return None

In [25]:
embedding = get_word_embedding('مرد', vocab, word2vec_model)
print(embedding)

[-0.11791629  0.09824163  0.16149867  0.36515609  0.27589637  0.33782451
 -0.33546593 -0.23761117 -0.37162012  0.09611822  0.46401785  0.21281058
 -0.05096967  0.39538357  0.22121418  0.21732541 -0.27843149 -0.34723499
  0.4329129   0.51446148  0.41265543  0.35946686  0.28507072 -0.16971806
 -0.33671245 -0.41374205 -0.05198899 -0.22776971 -0.2486474   0.43918947
 -0.38915345  0.49060989 -0.23385779 -0.12272904  0.00989349  0.27630257
 -0.19655076 -0.44353416 -0.27768833  0.41741545 -0.07189619  0.10620767
 -0.39507999 -0.08934812 -0.26004592 -0.48503162  0.04088924 -0.14932115
  0.33315881  0.2392398  -0.45263298  0.1381911  -0.42307376  0.11054123
  0.2718077  -0.46727458 -0.16071644 -0.06832454  0.13632549 -0.37259475
 -0.19520646  0.32874091 -0.01753422  0.15881387 -0.32520808 -0.00463953
  0.10513484  0.36069387 -0.2893219  -0.21934731 -0.24544643 -0.20634754
  0.24809312 -0.04809858 -0.35267489  0.50107476  0.40511108  0.20499646
 -0.05248257  0.47079717  0.36123785 -0.09024045 -0

In [28]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def find_similar_words(word, vocab, model, k = 5):
    word_idx = vocab.word_to_index(word)

    if word_idx == -1:
        return f"Word '{word}' not in vocabulary."

    word_vector = model.W[word_idx]
    similarities = []

    for idx in range(vocab.vocab_size):
        if idx != word_idx:
            other_word_vector = model.W[idx]
            sim_score = cosine_similarity(word_vector, other_word_vector)
            similarities.append((vocab.index_to_word(idx), sim_score))

    similarities = sorted(similarities, key = lambda x: x[1], reverse = True)

    return similarities[:k]

In [29]:
similar_words = find_similar_words("ملکه", vocab, word2vec_model, k = 5)
print(similar_words)

[('شاه', 0.08961007812423587), ('مرد', 0.08729523491489555), ('زن', 0.03178485519043798), ('کمک', -0.1395635367391647)]


In [32]:
def find_analogy(word_a, word_b, word_c, vocab, model):
    # Convert words to their vector representations
    vec_a = model.W[vocab.word_to_index(word_a)] if vocab.word_to_index(word_a) != -1 else None
    vec_b = model.W[vocab.word_to_index(word_b)] if vocab.word_to_index(word_b) != -1 else None
    vec_c = model.W[vocab.word_to_index(word_c)] if vocab.word_to_index(word_c) != -1 else None

    if vec_a is None or vec_b is None or vec_c is None:
        return f"One of the words '{word_a}', '{word_b}', or '{word_c}' is not in the vocabulary."

    # Word analogy: vector equation (vec_d = vec_a - vec_b + vec_c)
    target_vec = vec_a - vec_b + vec_c
    similarities = []

    for idx in range(vocab.vocab_size):
        other_word_vector = model.W[idx]
        sim_score = cosine_similarity(target_vec, other_word_vector)
        similarities.append((vocab.index_to_word(idx), sim_score))

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    filtered_similarities = [word for word in similarities if word[0] not in {word_a, word_b, word_c}]

    return filtered_similarities[0] if filtered_similarities else None

# Example usage:
analogy_result = find_analogy("مرد", "شاه", "زن", vocab, word2vec_model)
print(analogy_result)

('کمک', 0.09306850703985767)
